In [ ]:
from elevation_aware_ssl.classification import wandb_utils
import plotly.express as px

import wandb
#wandb.login(key="wandb_key")
wandb.login(key="wandb_key")

In [ ]:
select_classes = ["non_agricultural", "agricultural_frontier"]

In [ ]:
project = "Segmentation-2_classes"
entity = "omar-c"
name = "report"
version = ["RandomInit", "SimCLR", "Elevation"]
train_size = [10, 20, 40, 80, 100, 200, 500, 1000]

bar_plots = {}
for metric in ["IoU" ,"Recall", "Precision", "F1_score", "Acc_by_Class"]:
    plot = wandb_utils.lineplot_metrics_from_wandb(
        project=project,
        entity=entity,
        version=list(set(version)),
        train_size=list(set(train_size)),
        metric=f"{metric}",
    )

    bar_plots.update({metric: plot})

In [ ]:
fig = bar_plots["IoU"]
fig.update_layout(width=1200, height = 500)

In [ ]:
fig = bar_plots["F1_score"]
fig.update_layout(width=1200, height = 500)

In [ ]:
project = "Segmentation-2_classes"
entity = "omar-c"
name = "report"
version = ["RandomInit", "SimCLR", "Elevation"]
train_size = [100]#[10, 20, 40, 100, 200, 500, 1000]


bar_plots = {}
for metric in ["Recall", "Precision", "F1_score", "Acc_by_Class"]:
    plot = wandb_utils.barplot_metrics_from_wandb(
        project=project,
        entity=entity,
        version=list(set(version)),
        train_size=list(set(train_size)),
        metric=f"{metric}",
    )

    bar_plots.update({metric: plot})

In [ ]:
bar_plots["F1_score"]

In [ ]:
def standar_report(project, entity, name, version, train_size):

    bar_plots = {}
    for metric in ["Recall", "Precision", "F1_score", "Acc_by_Class"]:
        plot = wandb_utils.barplot_metrics_from_wandb(
            project=project,
            entity=entity,
            version=list(set(version)),
            train_size=list(set(train_size)),
            metric=f"{metric}",
        )

        bar_plots.update({metric: plot})

    line_plots = {}
    for metric in ["Recall", "Precision", "F1_score", "Acc_by_Class"]:
        plot = wandb_utils.lineplot_metrics_from_wandb(
            project=project,
            entity=entity,
            version=list(set(version)),
            train_size=list(set(train_size)),
            metric=f"{metric}",
        )

        line_plots.update({metric: plot})

    table = wandb_utils.get_table(
        project=project,
        entity=entity,
        version=version,
        train_size=train_size,
    )

    pr_curve = {}
    for label in select_classes:

        plot = wandb_utils.plot_pr_curve(table.query(f"label=='{label}'"), color="train_size", line_dash="version")

        pr_curve.update({label: plot})

    #loss_table = wandb_utils.get_table(
    #    wandb,
    #    project=project,
    #    entity=entity,
    #    version=version,
    #    train_size=train_size,
    #    table_name="Loss",
    #)

    #loss_plot = wandb_utils.plot_loss_curves(loss_table, color="train_size", line_dash="version")

    with wandb.init(
        project=project,
        entity=entity,
        config={"version": "report", "train_size": None},
        name=name,
    ) as run:

        #run.log({"Label Distribution": px.histogram(metadata, x="Classes", histnorm="percent")})
        [run.log({f"Bareplot {metric}": plot}) for metric, plot in bar_plots.items()]
        [run.log({f"Lineplot {metric}": plot}) for metric, plot in line_plots.items()]
        [run.log({f"PR Curve {label}": plot}) for label, plot in pr_curve.items()]
        #run.log({"Loss Plots": loss_plot})

In [ ]:
standar_report(
    project="CLF-2_classes",
    entity="omar_castano",
    name="report",
    version=["RandomInit"] * 8 + ["SimCLR"] * 8 + ["Elevation"] * 8,
    train_size=[10, 20, 40, 80, 100, 200, 500, 1000] * 3,
)